In [11]:
%load_ext autoreload
%autoreload 2

import os
import pickle
import numpy as np
import pandas as pd

import nd2
from tifffile import imread
from skimage.io import imread as imread_png

from skimage.measure import regionprops_table
import scipy.stats as stats
from scipy.stats import circmean, circstd
from scipy.ndimage import distance_transform_edt
from shapely.geometry import Polygon

from cardiomyocytes_helper_functions import segment_actin_3D, calculate_perpendicular_index, get_internal_points, create_edge_visual,find_fibers_orientation_v2, orientations_from_vertices,interpolate_and_fill

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
mask_dir = r'D:\data_analysis\2022_Sahana\OneDrive_1_20-06-2023\Images_with_DAPI\Masks'
vertices_dir = mask_dir
im_dir = r'C:\Users\lab\OneDrive - University of Pittsburgh\230516_reorder_Sahana\Images'

legend_path = r'D:\data_analysis\2022_Sahana\scrambler.csv'

save_path = r'D:\data_analysis\2022_Sahana\OneDrive_1_20-06-2023\Images_with_DAPI\Results_recalculated'

In [3]:
# read in legend how images were scrambled
df_legend = pd.read_csv(legend_path)

## Prepare for processing

In [4]:
def get_condition(secret_n):

    old_path = df_legend.loc[df_legend.num == secret_n,'old_path'].tolist()[0]

    if 'Fibronectin' in old_path:
        condition = 'Fibronectin'
    else:
        condition = 'Collagen'

    return condition

In [5]:
def check_dapi(num):

    im_path = os.path.join(im_dir,f'{str(num).zfill(2)}.nd2')
    im = nd2.imread(im_path)

    if im.shape[1]>2:
        dapi = True
    else:
        dapi = False
    
    return dapi

In [6]:
# order of channels in this set
channel_names = ['actin','plak','DAPI']

# general properties of cells to be calculated
properties = ['label','area','centroid','bbox','eccentricity','orientation','intensity_image','image','intensity_mean']

# number of points to re-sample the edge
point_num = 200

# proposed size of the hanning window for smoothing
# replaced with the 10% of signal length
hanning_size = 81

# number of layers for radial distributions
layer_num = 20

# original df columns
columns_org = ['label', 'area', 'centroid-0', 'centroid-1', 'bbox-0', 'bbox-1', 'bbox-2', 'bbox-3', 'eccentricity', 'orientation', 'intensity_image', 'image'] #note that image and intensity image are added in this orientation 
columns_intensity = [f'mean_{x}' for x in channel_names] + ['mean_fibers']

# prepare column names to store complex things later
columns_to_add = ['actin_detected','actin_angles','membrane_orientation','mem_act_orientation','edge_points','edge_image','edge_points_cell','mem_orientation_edge','plak_smooth','orientation_smooth']
# add names of edge signals based on channel names
for c in channel_names:
    columns_to_add.append(f'edge_signal_{c}')
# add names of radial signals based on channel names
for c in channel_names:
    columns_to_add.append(f'signal_radial_{c}')

In [7]:
# get a list of masks to process

file_list = [x for x in os.listdir(mask_dir) if 'png' in x]
file_list

['01_mask.png',
 '02_mask.png',
 '04_mask.png',
 '05_mask.png',
 '07_mask.png',
 '16_mask.png',
 '17_mask.png',
 '20_mask.png',
 '24_mask.png',
 '27_mask.png',
 '29_mask.png',
 '30_mask.png',
 '31_mask.png',
 '33_mask.png',
 '36_mask.png',
 '38_mask.png',
 '40_mask.png',
 '41_mask.png',
 '46_mask.png',
 '47_mask.png']

## Loop processing the images

In [12]:
data_list = []

for mask_file in file_list:

    ####################################################################################################
    # open image and mask

    # open mask 
    mask_path = os.path.join(mask_dir,mask_file)
    mask = imread_png(mask_path)

    # open original image
    num = int(mask_file[:2])
    im_name = str(num).zfill(2) +'.nd2'
    im_path = os.path.join(im_dir,im_name)
    im = nd2.imread(im_path)

    ####################################################################################################
    # read in vertices

    pkl_path = os.path.join(vertices_dir,im_name.replace('.nd2','_polygons.pkl'))

    with open(pkl_path, 'rb') as f:
        vertices_polygons = pickle.load(f)

    ####################################################################################################
    # actin segmentation

    # get actin channel
    image_actin = im[:,0,:,:]

    # segment actin volume
    image_actin_mask = segment_actin_3D(image_actin)

    # flatten segmented actin
    image_actin_mask_2D = np.max(image_actin_mask,axis=0)

    ####################################################################################################
    # calculate general properties
    im_flat_all_channels = np.max(im,axis=0)
    im_flat_all = np.append(im_flat_all_channels,np.expand_dims(image_actin_mask_2D,axis=0),axis=0)
    im_flat_all = np.moveaxis(im_flat_all,0,2)

    cell_measure = regionprops_table(mask, intensity_image = im_flat_all, properties = properties)
    df = pd.DataFrame(cell_measure)

    ####################################################################################################
    # maintenance of the df
    # renaming and adding columns
    # adding addtional info
    
    # renaming columns
    df.columns = columns_org + columns_intensity

    # adding columns
    for c in columns_to_add:
        df[c] = None
        df[c] = df[c].astype(object)

    # adding info to df
    df['image_name'] = im_name
    
    # uncover and add condition
    condition = get_condition(num)
    df['condition'] = condition

    # check that the image contain DAPI
    dapi = check_dapi(num)
    df['dapi_present'] = dapi

    ####################################################################################################
    # add vertices to the table
    df['vertices'] = vertices_polygons

    ####################################################################################################################################
    ####################################################################################################################################
    ####################################################################################################################################
    # add calculations for individual cells

    for ind,cell in df.iterrows():

        # ACTIN
        ####################################################################################################
        ####################################################################################################
        # calculate actin organization

        # calculate dominant actin orientation
        im_single_cell_actin = cell.intensity_image[:,:,-1]

        actin,actin_angles = find_fibers_orientation_v2(im_single_cell_actin)
        
        # calculate mean orientation of the fibers
        actin_orientation = circmean(actin_angles,low = -np.pi/2, high = np.pi/2)

        # calculate spread of orientations
        actin_spread = circstd(actin_angles,low = -np.pi/2, high = np.pi/2)

        df.at[ind,'actin_detected'] = np.array(actin)
        df.at[ind,'actin_angles'] = np.array(actin_angles)
        df.loc[ind,'actin_orientation'] = actin_orientation
        df.loc[ind,'actin_spread'] = actin_spread

        # RADIAL
        ####################################################################################################
        ####################################################################################################
        # calculate radial distributions

        # generate distance transform
        cell_shape = cell.image
        dist = distance_transform_edt(cell_shape)

        # digitize distance transform
        step = np.max(dist)/layer_num

        for n in range(layer_num):

            dist[(dist>(n*step)) & (dist<=((n+1)*step))] = n+1

        # calculate radial distribution signals
        for s,ch_name in enumerate(channel_names):

            signal_image = cell.intensity_image[:,:,s]

            signal_list = []
            for n in range(layer_num):

                signal = np.median(signal_image[dist==(n+1)])

                signal_list.append(signal)

            df.at[ind,f'signal_radial_{ch_name}'] = np.array(signal_list)

        # EDGE
        ####################################################################################################
        ####################################################################################################
        # add cell vertices
        vertices = (cell.vertices).astype(int)

        ####################################################################################################
        # calculate membrane orientation
        membrane_orientation = orientations_from_vertices(vertices)
        df.at[ind,'membrane_orientation'] = membrane_orientation

        ####################################################################################################
        # recalculate membrane orientation vs actin
        orientations = []
        for angle_membrane in membrane_orientation:

            orientation = calculate_perpendicular_index(actin_orientation,angle_membrane)

            orientations.append(orientation)

        df.at[ind,'mem_act_orientation'] = np.array(orientations)

        ####################################################################################################
        # calculate points at the perimeter

        # create a polygon from vertices
        p = Polygon(cell.vertices)

        # gen n points evently distributed at the border
        perim = np.array([p.exterior.interpolate(t).xy for t in np.linspace(0,p.length,point_num,False)])[:,:,0]
   
        df.at[ind,'edge_points'] = np.array(perim)

        ####################################################################################################
        # recalculate perimeter points in the coordinate system of a cell
        perim[:,0] = perim[:,0] - cell['bbox-0']
        perim[:,1] = perim[:,1] - cell['bbox-1']

        df.at[ind,'edge_points_cell'] = np.array(perim)

        ####################################################################################################
        # get points distributed at the periphery
        df_points = get_internal_points(np.array(perim),line_width=21)

        # clean points
        df_points['dr'] = [((x>=cell.image.shape[0]) or (y>=cell.image.shape[1])) for x,y in zip(df_points.r,df_points.c)]
        df_points.drop(df_points.index[df_points.dr],axis=0,inplace=True)

        ####################################################################################################
        # calculate edge signals from intensity channels
        for i,ch_name in enumerate(channel_names):

            # prepare image for signal calculation
            im = (cell.intensity_image[:,:,i] * cell.image).astype(float)
            im[im==0] = np.NaN

            # collect signal values from prepared image
            df_points['signal'] = [im[r,c] for r,c in zip(df_points.r,df_points.c)]

            # group by order of points
            res = df_points.loc[:,['ord','signal']].groupby('ord').mean().reset_index()

            # store signal values
            c_name = f'edge_signal_{ch_name}'
            df.at[ind,c_name] = np.array(res)[:,1]


        ####################################################################################################
        # calculate edge orientation at edge_points

        vertices = cell.vertices.copy()
        vertices[:,0] = [np.min([x,im.shape[0]-1]) for x in vertices[:,0] - cell['bbox-0']]
        vertices[:,1] = [np.min([x,im.shape[1]-1]) for x in vertices[:,1] - cell['bbox-1']]
        vertices = vertices.astype(int)

        orientations = np.array(orientations)

        # prepare image of edge orientation vs actin
        edge_image = create_edge_visual(cell.image.shape,vertices,orientations,line_width=5)
        edge_image[edge_image==0] = np.NaN

        df.at[ind,'edge_image'] = edge_image

        # collect orientaton values from prepared image
        df_points['signal'] = [edge_image[r,c] for r,c in zip(df_points.r,df_points.c)]

        # group by order of points
        res = df_points.loc[:,['ord','signal']].groupby('ord').mean().reset_index()

        # store signal values
        df.at[ind,'mem_orientation_edge'] = np.array(res)[:,1]

        ####################################################################################################
        # Calculate smooth signals
        # w = np.hanning(hanning_size)

        plak_signal = df.loc[ind,'edge_signal_plak']
        plak_signal = interpolate_and_fill(plak_signal)

        w = np.hanning(int(len(plak_signal)/10))

        plak_smooth = np.convolve(w/w.sum(),plak_signal,mode='same')
        df.at[ind,'plak_smooth'] = np.array(plak_smooth)

        orientation_signal = df.loc[ind,'mem_orientation_edge']
        orientation_signal = interpolate_and_fill(orientation_signal)
        orientation_smooth = np.convolve(w/w.sum(),orientation_signal,mode='same')
        df.at[ind,'orientation_smooth'] = np.array(orientation_smooth)

        ####################################################################################################
        # calculate Pearson correlation between edge orientation and plakoglobin signal
        r, p = stats.pearsonr(orientation_smooth, plak_smooth)
        df.loc[ind,'Pearson'] = r

    # save data frame
    df.to_pickle(os.path.join(save_path,im_name.replace('.nd2','_df_pear.pkl')))
    data_list.append(df)

intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absolutein